In [1]:
import pyspark as ps

In [2]:
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("sparkSQL exercise") 
        .getOrCreate()
        )

In [3]:
sc = spark.sparkContext

In [4]:
ls

transactions.txt  users.txt
Untitled.ipynb    yelp_academic_dataset_business.json.gz


In [8]:
df =spark.read.json('yelp_academic_dataset_business.json.gz')

In [9]:
df.show(5)

+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+----------+-----------+--------------------+-------------+-----+------------+-----+-----+--------+
|          attributes|         business_id|          categories|       city|        full_address|               hours|  latitude|  longitude|                name|neighborhoods| open|review_count|stars|state|    type|
+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+----------+-----------+--------------------+-------------+-----+------------+-----+-----+--------+
|[,,,,,,,, true,,,...|vcNAWiLM4dR7D2nww...|[Doctors, Health ...|    Phoenix|4840 E Indian Sch...|[[17:00, 08:00], ...| 33.499313|-111.983758|   Eric Goldberg, MD|           []| true|           9|  3.5|   AZ|business|
|[true,,,,,,,,,,,,...|UsFtqoBl7naz8AVUB...|         [Nightlife]| Dravosburg|202 McClure St
Dr...|            [,,,,,,]| 40.350519|  -

In [10]:
df.printSchema()

root
 |-- attributes: struct (nullable = true)
 |    |-- Accepts Credit Cards: string (nullable = true)
 |    |-- Accepts Insurance: boolean (nullable = true)
 |    |-- Ages Allowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: struct (nullable = true)
 |    |    |-- casual: boolean (nullable = true)
 |    |    |-- classy: boolean (nullable = true)
 |    |    |-- divey: boolean (nullable = true)
 |    |    |-- hipster: boolean (nullable = true)
 |    |    |-- intimate: boolean (nullable = true)
 |    |    |-- romantic: boolean (nullable = true)
 |    |    |-- touristy: boolean (nullable = true)
 |    |    |-- trendy: boolean (nullable = true)
 |    |    |-- upscale: boolean (nullable = true)
 |    |-- Attire: string (nullable = true)
 |    |-- BYOB: boolean (nullable = true)
 |    |-- BYOB/Corkage: string (nullable = true)
 |    |-- By Appointment Only: boolean (nullable = true)
 |    |-- Caters: boolean (nullable = true)
 |    |-- Coat Check

In [11]:
df.createOrReplaceTempView('yelp_business')

In [12]:
df.show()

+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+----------+-----------+--------------------+-------------+-----+------------+-----+-----+--------+
|          attributes|         business_id|          categories|       city|        full_address|               hours|  latitude|  longitude|                name|neighborhoods| open|review_count|stars|state|    type|
+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+----------+-----------+--------------------+-------------+-----+------------+-----+-----+--------+
|[,,,,,,,, true,,,...|vcNAWiLM4dR7D2nww...|[Doctors, Health ...|    Phoenix|4840 E Indian Sch...|[[17:00, 08:00], ...| 33.499313|-111.983758|   Eric Goldberg, MD|           []| true|           9|  3.5|   AZ|business|
|[true,,,,,,,,,,,,...|UsFtqoBl7naz8AVUB...|         [Nightlife]| Dravosburg|202 McClure St
Dr...|            [,,,,,,]| 40.350519|  -

In [28]:
result = spark.sql("SELECT name, city, state, stars, attributes['Accepts Credit Cards'] FROM yelp_business LIMIT 10")
result.show()

+--------------------+-----------+-----+-----+-------------------------------+
|                name|       city|state|stars|attributes.Accepts Credit Cards|
+--------------------+-----------+-----+-----+-------------------------------+
|   Eric Goldberg, MD|    Phoenix|   AZ|  3.5|                           null|
|        Clancy's Pub| Dravosburg|   PA|  3.5|                           true|
|Cool Springs Golf...|Bethel Park|   PA|  2.5|                           null|
|    Verizon Wireless| Pittsburgh|   PA|  3.5|                           null|
|       Emil's Lounge|   Braddock|   PA|  4.5|                           true|
|Alexion's Bar & G...|   Carnegie|   PA|  4.0|                           true|
|Flynn's E W Tire ...|   Carnegie|   PA|  1.5|                           null|
|Forsythe Miniatur...|   Carnegie|   PA|  4.0|                           null|
|Quaker State Cons...|   Carnegie|   PA|  2.5|                           null|
|Kings Family Rest...|   Carnegie|   PA|  3.5|      

- Rated at 5 stars
- In the city of Phoenix
- Accepts credit card (Reference the 'Accepts Credit Card' field by attributes.`Accepts Credit Cards`. NOTE: We are actually looking for the value 'true', not the boolean value True!)
- Contains Restaurants in the categories array.
- Hint 1 : LATERAL VIEW explode() can be used to access the individual elements of an array (i.e. the categories array). For reference, you can see the first example on this page.
- Hint 2: In spark, while using filter() or where(), you can create a condition that tests if a column, made of an array, contains a given value. The functions is pyspark.sql.functions.array_contains.

In [31]:
query = '''SELECT name, city, stars, attributes['Accepts Credit Cards'], categories
            FROM yelp_business
            Where stars=5.0 AND city = 'Phoenix' AND attributes['Accepts Credit Cards']='true'
 '''

small_table = spark.sql(query)
small_table.show()

+--------------------+-------+-----+-------------------------------+--------------------+
|                name|   city|stars|attributes.Accepts Credit Cards|          categories|
+--------------------+-------+-----+-------------------------------+--------------------+
|  DeChance & Company|Phoenix|  5.0|                           true|[Shopping, Jewelr...|
|     Sunshine Market|Phoenix|  5.0|                           true|     [Food, Grocery]|
|Conroy's Wholesal...|Phoenix|  5.0|                           true|[Flowers & Gifts,...|
|Blue Collar Clothing|Phoenix|  5.0|                           true|[Arts & Crafts, U...|
|        Sun Cellular|Phoenix|  5.0|                           true|[Local Services, ...|
| Umom New Day Center|Phoenix|  5.0|                           true|[Local Services, ...|
|       See's Candies|Phoenix|  5.0|                           true|[Food, Specialty ...|
|Movies On Central...|Phoenix|  5.0|                           true|[Videos & Video G...|
|Lovato's 

In [34]:
from pyspark.sql.functions import array_contains

In [42]:
filtered = small_table.where(array_contains("categories", "Restaurants"))

In [43]:
filtered.show()

+--------------------+-------+-----+-------------------------------+--------------------+
|                name|   city|stars|attributes.Accepts Credit Cards|          categories|
+--------------------+-------+-----+-------------------------------+--------------------+
|       Auslers Grill|Phoenix|  5.0|                           true|       [Restaurants]|
|Mulligan's Restau...|Phoenix|  5.0|                           true|       [Restaurants]|
|             Sunfare|Phoenix|  5.0|                           true|[Food Delivery Se...|
|              Subway|Phoenix|  5.0|                           true|[Fast Food, Sandw...|
|           Lil Cal's|Phoenix|  5.0|                           true|       [Restaurants]|
|                Ed's|Phoenix|  5.0|                           true|[American (Tradit...|
|Frenchys Caribbea...|Phoenix|  5.0|                           true|[Food, Hot Dogs, ...|
|           WY Market|Phoenix|  5.0|                           true|[American (Tradit...|
|       Po

## Advanced

In [44]:
users = spark.read.csv('users.txt',
                      header = False,
                      sep = ';',
                      inferSchema = True)



In [45]:
users.show(5)

+----------+-----------------+--------------------+--------------------+
|       _c0|              _c1|                 _c2|                 _c3|
+----------+-----------------+--------------------+--------------------+
|1106214172|Prometheus Barwis|prometheus.barwis...|      (533) 072-2779|
| 527133132|Ashraf Bainbridge|ashraf.bainbridge...|                null|
|1290614884|   Alain Hennesey|alain.hennesey@fa...|(942) 208-8460,(8...|
|1700818057| Hamed Fingerhuth|hamed.fingerhuth@...|                null|
|  17378782|    Annamae Leyte|annamae.leyte@msn...|                null|
+----------+-----------------+--------------------+--------------------+
only showing top 5 rows



In [46]:
users.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



In [ ]:
users= spark.read.csv('users.txt',
                      header = False,
                      sep = ';',
                      inferSchema = True)


In [101]:
transactions_rdd = sc.textFile('transactions.txt')

In [102]:
transactions_rdd.take(5)

['815581247;$144.82;2015-09-05',
 '1534673027;$140.93;2014-03-11',
 '842468364;$104.26;2014-05-06',
 '1720001139;$194.60;2015-08-24',
 '1397891675;$307.72;2015-09-25']

In [75]:
transactions_rdd =(sc.textFile('transactions.txt')
                   .map(lambda rowstr : rowstr.split(";"))
                   #.map(lambda rowstr: rowstr[1].replace("$",''))
                  )

In [76]:
transactions_rdd.take(5)

[['815581247', '$144.82', '2015-09-05'],
 ['1534673027', '$140.93', '2014-03-11'],
 ['842468364', '$104.26', '2014-05-06'],
 ['1720001139', '$194.60', '2015-08-24'],
 ['1397891675', '$307.72', '2015-09-25']]

In [ ]:
def casting_function(row):
    (id, date, store, state, product, amount) = row
    return (int(id), date, int(store), state, int(product), float(amount))
rdd_sales = (
    sc.textFile('data/sales.csv')
        .map(lambda rowstr : rowstr.split(","))
        .filter(lambda row: not row[0].startswith('#'))
        .map(casting_function)
            )
rdd_sales.collect()

In [97]:
def remove_dsign(row):
    row[1] = row[1].replace('$','')
    return row

def casting_function(row):
    (trans_id, cost, date) = row
    return (int(trans_id), float(cost), date)

transactions_rdd =(sc.textFile('transactions.txt')
                   .map(lambda rowstr : rowstr.split(";"))
                   .map(remove_dsign)
                   .map(casting_function)
                  )

transactions_rdd.take(5)

[(815581247, 144.82, '2015-09-05'),
 (1534673027, 140.93, '2014-03-11'),
 (842468364, 104.26, '2014-05-06'),
 (1720001139, 194.6, '2015-08-24'),
 (1397891675, 307.72, '2015-09-25')]

In [83]:
from pyspark.sql.types import *

In [98]:
# create a schema of your own
schema = StructType([
    StructField('trans_id',     IntegerType(), True),
    StructField('cost',  FloatType(), True),
    StructField('date',   StringType(),  True)
 ])
# feed that into a DataFrame
df = spark.createDataFrame(transactions_rdd, schema)

In [92]:
df.printSchema()

root
 |-- trans_id: integer (nullable = true)
 |-- cost: float (nullable = true)
 |-- date: string (nullable = true)



In [93]:
users.show(5)

+----------+-----------------+--------------------+--------------------+
|       _c0|              _c1|                 _c2|                 _c3|
+----------+-----------------+--------------------+--------------------+
|1106214172|Prometheus Barwis|prometheus.barwis...|      (533) 072-2779|
| 527133132|Ashraf Bainbridge|ashraf.bainbridge...|                null|
|1290614884|   Alain Hennesey|alain.hennesey@fa...|(942) 208-8460,(8...|
|1700818057| Hamed Fingerhuth|hamed.fingerhuth@...|                null|
|  17378782|    Annamae Leyte|annamae.leyte@msn...|                null|
+----------+-----------------+--------------------+--------------------+
only showing top 5 rows



In [100]:
df.show(5)

+----------+------+----------+
|  trans_id|  cost|      date|
+----------+------+----------+
| 815581247|144.82|2015-09-05|
|1534673027|140.93|2014-03-11|
| 842468364|104.26|2014-05-06|
|1720001139| 194.6|2015-08-24|
|1397891675|307.72|2015-09-25|
+----------+------+----------+
only showing top 5 rows



In [ ]:
df.createOrReplaceTempView('df')

In [104]:
query = '''SELECT _c1 as name, cost 
            FROM users
            RIGHT JOIN df ON users._c0 = df.trans_id
            ORDER BY cost DESC
            LIMIT 10
 '''

top_10 = spark.sql(query)
top_10.show()

AnalysisException: Table or view not found: users; line 2 pos 17;
'GlobalLimit 10
+- 'LocalLimit 10
   +- 'Sort ['cost DESC NULLS LAST], true
      +- 'Project ['_c1 AS name#707, 'cost]
         +- 'Join RightOuter, ('users._c0 = 'df.trans_id)
            :- 'UnresolvedRelation [users]
            +- 'UnresolvedRelation [df]
